In [ ]:
import mne, fooof, numpy as np, pandas as pd, os, glob, sys
from IPython.display import display, Markdown
from tqdm import tqdm, trange
import matplotlib.pyplot as plt

In [ ]:
datapath = "/nethome/kcni/moveisi/lab/repos/bme1500/po_scratch/data/bme1500_eeg_data"
os.chdir(datapath)

In [ ]:
def finder():
    filesDepth5 = glob.glob('*/*/*/*/*')
    dirsDepth5 = filter(lambda f: os.path.isdir(f), filesDepth5)

    markers=np.sort(np.array([os.path.join(datapath, f) for f in filesDepth5 if f.endswith('.pkl')]))
    npzs=np.sort(np.array([os.path.join(datapath, f) for f in filesDepth5 if f.endswith('.npz')]))
    return markers, npzs

In [ ]:
markers, npzs = finder()

In [ ]:
word_array = np.array(['/0/', '/1/','/15/','/16/', '/2/','/20/','/21/','/22/','/23/','/25/','/26/', '/27/','/28/','/3/','/5/','/6/'])
score_list = []
intensity_list = []
subjectID = []
for c in range(0,len(word_array)):
    score = []
    intensity = []
    print(word_array[c])
    for t in range (0,len(markers)):
        line = markers[t]
        if word_array[c] in line:
            _marker=pd.read_pickle(markers[t])
            score.append(_marker['clinScore'])
            intensity.append(_marker['stimInt'])
    score_list.append(score)
    intensity_list.append(intensity)
    subjectID.append(word_array[c])

In [ ]:
num_patient = len(intensity_list)
final_intensity =  [[] for _ in range(num_patient)]
final_score = [[] for _ in range(num_patient)]
for j in range(0,num_patient):
    for pos,i in enumerate(intensity_list[j]):
        arr = np.array(intensity_list[j])
        res = np.where(arr==i)[0]
        if len(res)==1:
            final_intensity[j].append(i)
            final_score[j].append(score_list[j][pos])
        else:
            exist = final_intensity[j].count(i)
            if exist == 0:
                temp = []
                for k in res:
                    temp.append(score_list[j][k])
                final_score[j].append(mean(temp))
                final_intensity[j].append(i)

In [ ]:
plt.figure(figsize=(20,10), dpi=300)
plt.rcParams["figure.figsize"] = (20,10)
for i in range(0,num_patient):
    plt.subplot(4,4,i+1)
    plt.scatter(final_intensity[i],final_score[i])
    s = subjectID[i]
    value = (s[s.find(start)+len(start):s.rfind(end)])
    plt.title('Subject ID: %s' %value)
    plt.ylabel('Score')
    plt.xlabel('Intensity')
      
plt.tight_layout()
plt.savefig('test.png')